## Background

In this article, we will use [softmax](https://en.wikipedia.org/wiki/Softmax_function) classifier to build a simple image classification neural network with an accuracy of 32%. In a Softmax classifier, binary logic is generalized and regressed to multiple logic. Softmax classifier will output the probability of the corresponding category.

We will first define a softmax classifier, then use the training set of [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) to train the neural network, and finally use the test set to verify the accuracy of the neural network.

Let’s get started.

## Import dependencies

Like the previous course [GettingStarted](https://thoughtworksinc.github.io/DeepLearning.scala/demo/GettingStarted.html), we need to introduce each class of DeepLearning.scala.

In [12]:
import $plugin.$ivy.`org.scalamacros:paradise_2.11.11:2.1.0`
import $ivy.`com.thoughtworks.deeplearning::jupyter-differentiable:2.0.0-M1`
import $ivy.`org.nd4j:nd4j-native-platform:0.7.2`
import $ivy.`org.rauschig:jarchivelib:0.5.0`
import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.2`

import java.io.{FileInputStream, InputStream}

import com.thoughtworks.deeplearning.math._
import com.thoughtworks.deeplearning.jupyter.differentiable.Any._
import com.thoughtworks.deeplearning.jupyter.differentiable.INDArray.{
  Optimizer => INDArrayOptimizer
}
import INDArrayOptimizer.LearningRate
import com.thoughtworks.deeplearning.jupyter.differentiable.INDArray.implicits._
import com.thoughtworks.each.Monadic._
import com.thoughtworks.raii.asynchronous.Do
import com.thoughtworks.deeplearning.jupyter.differentiable.Double._
import com.thoughtworks.deeplearning.jupyter.differentiable.Double.implicits._
import com.thoughtworks.deeplearning.Tape
import com.thoughtworks.deeplearning.jupyter.differentiable
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4s.Implicits._
import scala.concurrent.ExecutionContext.Implicits.global
import scalaz.concurrent.Task
import scalaz.{-\/, \/, \/-}
import scalaz.std.vector._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq

import $plugin.$                                            

import $ivy.$                                                               

import $ivy.$                                    

import $ivy.$                               

import $ivy.$                                             


import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning.math._

import com.thoughtworks.deeplearning.jupyter.differentiable.Any._

import com.thoughtworks.deeplearning.jupyter.differentiable.INDArray.{
  Optimizer => INDArrayOptimizer
}

import INDArrayOptimizer.LearningRate

import com.thoughtworks.deeplearning.jupyter.differentiable.INDArray.implicits._

import com.thoughtworks.each.Monadic._

import com.thoughtworks.raii.asynchronous.Do

import com.thoughtworks.deeplearning.jupyter.differentiable.Double._

import com.thoughtworks.deeplearning.jupyter.differentiable.Double.implicits._

import com.thoughtworks.deeplearning.Tape

import com.thoughtworks.deeplearnin

To reduce the line numbers outputted by `jupyter-scala` and to make sure that the page output will not be too long, we need to set `pprintConfig`.

In [13]:
pprintConfig() = pprintConfig().copy(height = 2)

## Build your own neural network.

### Set learning rate

Learning rate need to be set for the full connection layer. Learning rate visually describes the change rate of `weight`. A too-low learning rate will result in slow decrease of `loss`, which will require longer time for training; A too-high learning rate will result in rapid decrease of `loss` at first while fluctuation around the lowest point afterward.

In [14]:
implicit def optimizer: INDArrayOptimizer = new LearningRate {
  def currentLearningRate() = 0.00001
}

defined function optimizer

### Write softmax

To use `softmax` classifier (softmax classifier is a neural network combined by `softmax` and a full connection), we first need to write softmax function, formula: ![](https://www.zhihu.com/equation?tex=f_j%28z%29%3D%5Cfrac%7Be%5E%7Bz_j%7D%7D%7B%5Csum_ke%5E%7Bz_k%7D%7D)

In [15]:
def softmax(scores: differentiable.INDArray): differentiable.INDArray = {
  val expScores = exp(scores)
  expScores / sum(expScores, 1)
}

defined function softmax

### Compose your  neural network

Define a full connection layer and [initialize Weight](https://github.com/ThoughtWorksInc/DeepLearning.scala/wiki/Getting-Started#231--weight-intialization), `Weight` shall be a two-dimension `INDArray` of `NumberOfPixels × NumberOfClasses`. `scores` is the score of each image corresponding to each category, representing the feasible probability of each category corresponding to each image.

In [16]:
//10 label of CIFAR10 images(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10
val NumberOfPixels: Int = 3072

val weight: differentiable.INDArray =
  (Nd4j.randn(NumberOfPixels, NumberOfClasses) * 0.001).toWeight

def myNeuralNetwork(input: INDArray): differentiable.INDArray = {
  softmax(dot(input, weight))
}

NumberOfClasses: Int = 10
NumberOfPixels: Int = 3072
weight: differentiable.package.INDArray = Suspend(<function0>)
defined function myNeuralNetwork

### Create LossFunction

To learn about the prediction result of the neural network, we need to write the loss function `lossFunction`. We use [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy) to make comparison between this result and the actual result before return the score. Formula:
![](https://zhihu.com/equation?tex=%5Cdisplaystyle+H%28p%2Cq%29%3D-%5Csum_xp%28x%29+logq%28x%29)

In [17]:
def lossFunction(input: INDArray,
                 expectOutput: INDArray): differentiable.Double = {
  val probabilities = myNeuralNetwork(input)
  -mean(log(probabilities) * expectOutput)
}

defined function lossFunction

## Prepare data

## Read data

To read the images and corresponding label information for test data from CIFAR10 database and process them, we need [`import $file.ReadCIFAR10ToNDArray`](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/ReadCIFAR10ToNDArray.sc). This is a script file containing the read and processed CIFAR10 data, provided in this course.

In [18]:
import $url.{`https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/master/ipynbs/ReadCIFAR10ToNDArray.sc` => ReadCIFAR10ToNDArray}

val trainNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/data_batch_1.bin", 1000)

val testNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

Compiling https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/master/ipynbs/ReadCIFAR10ToNDArray.sc


import $url.$                                                                                                                                             


trainNDArray: shapeless.::[INDArray, shapeless.::[INDArray, shapeless.HNil]] = [[0.23, 0.17, 0.20, 0.27, 0.38, 0.46, 0.54, 0.57, 0.58, 0.58, 0.51, 0.49, 0.55, 0.56, 0.54, 0.50, 0.54, 0.52, 0.48, 0.54, 0.54, 0.52, 0.53, 0.54, 0.59, 0.64, 0....
testNDArray: shapeless.::[INDArray, shapeless.::[INDArray, shapeless.HNil]] = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....

### Process data

Before passing data to the softmax classifier, we first process label data with ([one hot encoding](https://en.wikipedia.org/wiki/One-hot)): transform INDArray of `NumberOfPixels × 1` into INDArray of `NumberOfPixels × NumberOfClasses`. The value of correct classification corresponding to each line is 1, and the values of other columns are 0. The reason for differentiating the training set and test set is to make it clear that whether the network is over trained which leads to [overfitting](https://en.wikipedia.org/wiki/Overfitting). While processing label data, we used [Utils](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/Utils.sc), which is also provided in this course.

In [19]:
val trainData = trainNDArray.head
val testData = testNDArray.head

val trainExpectResult = trainNDArray.tail.head
val testExpectResult = testNDArray.tail.head

import $url.{`https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/master/ipynbs/Utils.sc` => Utils}

val vectorizedTrainExpectResult = Utils.makeVectorized(trainExpectResult, NumberOfClasses)
val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

Compiling https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/master/ipynbs/Utils.sc


trainData: INDArray = [[0.23, 0.17, 0.20, 0.27, 0.38, 0.46, 0.54, 0.57, 0.58, 0.58, 0.51, 0.49, 0.55, 0.56, 0.54, 0.50, 0.54, 0.52, 0.48, 0.54, 0.54, 0.52, 0.53, 0.54, 0.59, 0.64, 0....
testData: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
trainExpectResult: INDArray = [6.00, 9.00, 9.00, 4.00, 1.00, 1.00, 2.00, 7.00, 8.00, 3.00, 4.00, 7.00, 7.00, 2.00, 9.00, 9.00, 9.00, 3.00, 2.00, 6.00, 4.00, 3.00, 6.00, 6.00, 2.00, 6.00, 3.0...
testExpectResult: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00, 5.00, 7.00, 9.00, 8.00, 5.00, 7.00, 8.00, 6.00, 7.00, 0.00, 4.00, 9.00, 5.00, 2.00, 4.0...
import $url.$                                                                                                               


vectorizedTrainExpectResult: INDArray = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00],
 [0.00, 0.00

## Train your neural network

To observe the training process of the neural network, we need to output `loss`; while training the neural network, the `loss` shall be deceasing.

In [20]:
plotly.JupyterScala.init()
def polyLoss(lossSeq: IndexedSeq[Double]): Unit = {
  plotly.JupyterScala.init()

  val plot = Seq(
    Scatter(lossSeq.indices, lossSeq)
  )

  plot.plot(
    title = "loss by time"
  )
}

@monadic[Task]
val trainTask: Task[Unit] = {

  val lossSeq = for (_ <- (1 to 2000).toVector) yield {
    train(lossFunction(trainData, vectorizedTrainExpectResult)).each
  }

  polyLoss(lossSeq)
}

defined function polyLoss
trainTask: Task[Unit] = scalaz.concurrent.Task@99d9545

## Predict  your Neural Network

We use the processed test data to verify the prediction result of the neural network and compute the accuracy. The accuracy shall be about 32%.

In [21]:
val predictResult = throwableMonadic[Task] {
  trainTask.each
  predict(myNeuralNetwork(testData)).each
}

predictResult: Task[Tape.<refinement>.this.type.Data] = scalaz.concurrent.Task@6916cc60

In [22]:
predictResult.unsafePerformSyncAttempt match {
  case -\/(e) => {
    throw e
  }
  case \/-(result) =>
    println("The accuracy is " + Utils.getAccuracy(result,testExpectResult) + "%")
}

The accuracy is 32.0%


## Summary

We have learned the follows in this article:

* Prepare and process CIFAR10 data
* Write softmax classifier
* Use the prediction image of the neural network written by softmax classifier to match with the probability of each category.

[Source code](https://github.com/izhangzhihao/deeplearning-tutorial/blob/2.0.x/src/main/scala/com/github/izhangzhihao/SoftmaxLinearClassifier.scala)